In [0]:
from google.colab import drive
drive.mount('/content/drive')

import re

def get_title(name):
    title_search = re.search(' ([A-Za-z]+)\.', name)
    if title_search:
        return title_search.group(1)
    return ""

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import zipfile
z = zipfile.ZipFile('/content/drive/My Drive/titanic.zip', 'r')
z.extractall()

In [0]:
!ls

drive  gender_submission.csv  sample_data  test.csv  train.csv


In [0]:
import pandas as pd

train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

In [0]:
train_data.head(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [0]:
#train_data['Ticket'][:50]

In [0]:
train_data['Has_Cabin'] = train_data["Cabin"].apply(lambda x: 0 if type(x) == float else 1)
train_data['FamilySize'] = train_data['SibSp'] + train_data['Parch'] + 1
train_data['Is_alone'] = train_data["FamilySize"].apply(lambda x: 0 if x > 1 else 1)

# Работа с именем --------------------------------------------------------------------
train_data['Title'] = train_data['Name'].apply(get_title)
train_data['Title'] = train_data['Title'].replace(['Lady', 'Countess','Capt', 'Col','Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')
train_data['Title'] = train_data['Title'].replace('Mlle', 'Miss')
train_data['Title'] = train_data['Title'].replace('Ms', 'Miss')
train_data['Title'] = train_data['Title'].replace('Mme', 'Mrs')

title_mapping = {"Mr": 1, "Master": 2, "Mrs": 3, "Miss": 4, "Rare": 5}
train_data['Title'] = train_data['Title'].map(title_mapping)
train_data['Title'] = train_data['Title'].fillna(0)
# Работа с именем --------------------------------------------------------------------

In [0]:
test_data['Has_Cabin'] = test_data["Cabin"].apply(lambda x: 0 if type(x) == float else 1)
test_data['FamilySize'] = test_data['SibSp'] + test_data['Parch'] + 1
test_data['Is_alone'] = test_data["FamilySize"].apply(lambda x: 0 if x > 1 else 1)

# Работа с именем --------------------------------------------------------------------
test_data['Title'] = test_data['Name'].apply(get_title)
test_data['Title'] = test_data['Title'].replace(['Lady', 'Countess','Capt', 'Col','Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')
test_data['Title'] = test_data['Title'].replace('Mlle', 'Miss')
test_data['Title'] = test_data['Title'].replace('Ms', 'Miss')
test_data['Title'] = test_data['Title'].replace('Mme', 'Mrs')


test_data['Title'] = test_data['Title'].map(title_mapping)
test_data['Title'] = test_data['Title'].fillna(0)
# Работа с именем --------------------------------------------------------------------

In [0]:
train_data = train_data.drop(['PassengerId','Name','Ticket','Cabin'], axis = 1)

In [0]:
passenger_id = test_data['PassengerId']
test_data = test_data.drop(['PassengerId','Name','Ticket','Cabin'], axis = 1)

In [0]:
train_data.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Has_Cabin,FamilySize,Is_alone,Title
0,0,3,male,22.0,1,0,7.2500,S,0,2,0,1
1,1,1,female,38.0,1,0,71.2833,C,1,2,0,3
2,1,3,female,26.0,0,0,7.9250,S,0,1,1,4
3,1,1,female,35.0,1,0,53.1000,S,1,2,0,3
4,0,3,male,35.0,0,0,8.0500,S,0,1,1,1


**Перекодировка**

In [0]:
map_dict_sex = {'male': 1 , 'female': 0}
#print(map_dict_sex)
train_data['Sex'] = train_data['Sex'].map(map_dict_sex)

In [0]:
test_data['Sex'] = test_data['Sex'].map(map_dict_sex)

In [0]:
train_data['Embarked'].value_counts()

S    644
C    168
Q     77
Name: Embarked, dtype: int64

In [0]:
map_dict_embarked = {'S': 0 , 'C': 1, 'Q':2}

train_data['Embarked'] = train_data['Embarked'].map(map_dict_embarked)

In [0]:
test_data['Embarked'] = test_data['Embarked'].map(map_dict_embarked)

In [0]:
# Среднее арифметическое для возраста
train_data['Age']  = train_data['Age'].fillna(value = train_data['Age'].mean())
train_data['Embarked']  = train_data['Embarked'].fillna(value = 0)

In [0]:
test_data['Age']  = test_data['Age'].fillna(value = test_data['Age'].mean())
test_data['Fare']  = test_data['Fare'].fillna(value = test_data['Fare'].median())

In [0]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
Survived      891 non-null int64
Pclass        891 non-null int64
Sex           891 non-null int64
Age           891 non-null float64
SibSp         891 non-null int64
Parch         891 non-null int64
Fare          891 non-null float64
Embarked      891 non-null float64
Has_Cabin     891 non-null int64
FamilySize    891 non-null int64
Is_alone      891 non-null int64
Title         891 non-null int64
dtypes: float64(3), int64(9)
memory usage: 83.7 KB


In [0]:
test_data.info() # для выкладки решения на KAGGLE

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
Pclass        418 non-null int64
Sex           418 non-null int64
Age           418 non-null float64
SibSp         418 non-null int64
Parch         418 non-null int64
Fare          418 non-null float64
Embarked      418 non-null int64
Has_Cabin     418 non-null int64
FamilySize    418 non-null int64
Is_alone      418 non-null int64
Title         418 non-null int64
dtypes: float64(2), int64(9)
memory usage: 36.0 KB


In [0]:
y = train_data['Survived']
data = train_data
del data['Survived']

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, y, test_size=0.25, random_state=42)

In [0]:
# ПОСТРОЕНИЕ МОДЕЛИ!
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier(max_depth = 6, min_samples_split=5, random_state=0)
#clf.fit(X_train, y_train)
clf.fit(data, y)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=6, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=5,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=0, splitter='best')

In [0]:
y_predict = clf.predict(X_test)

In [0]:
y_predict[:20]

In [0]:
y_test[:20]

In [0]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_predict)
#0.8026905829596412 - было до has_cabin

0.8161434977578476

In [0]:
from sklearn.tree import export_graphviz

export_graphviz(clf, feature_names= ['Pclass','Sex','Age', 'SibSp','Parch','Fare','Embarked'], 
out_file='tree.dot', filled=True)

In [0]:
!ls

drive  gender_submission.csv  sample_data  test.csv  train.csv


In [0]:
!dot -Tpng 'tree.dot' -o 'tree.png'

In [0]:
from google.colab import files
files.download('tree.png')

In [0]:
predict = clf.predict(test_data)
submit = pd.DataFrame({'PassengerId':passenger_id,'Survived':predict})

In [0]:
#creating submission file
filename = 'Titanic Prediction.csv'
submit.to_csv(filename,index=False)
print('Saved file: ' + filename)

Saved file: Titanic Prediction.csv


In [0]:
!ls

 drive			 sample_data  'Titanic Prediction.csv'
 gender_submission.csv	 test.csv      train.csv


In [0]:
from google.colab import files
files.download('Titanic Prediction.csv')